In [1]:
import pandas as pd
import os


In [2]:
def build_advanced_dataset(data_dir, start_year=2018, end_year=2024):
    adv_list = []
    std_list = []

    for year in range(start_year, end_year + 1):

        # --- Advanced passing ---
        adv_files = [f for f in os.listdir(data_dir) if f"{year}_Playoffs_Advanced" in f]
        if not adv_files:
            print(f"No advanced playoff file found for {year}, skipping")
            continue
        adv_file = adv_files[0]

        # Read with two-level header
        adv_df = pd.read_html(os.path.join(data_dir, adv_file), header=[0,1])[0]

        # Drop all columns under RPO or Play Action
        drop_sections = ['RPO', 'PlayAction']
        cols_to_drop = [col for col in adv_df.columns if col[0] in drop_sections]
        adv_df = adv_df.drop(columns=cols_to_drop)

        # Flatten MultiIndex header → single row of column names
        adv_df.columns = [
            (c[1] if c[1] not in ['', None] else c[0]) 
            for c in adv_df.columns
        ]
        adv_df.columns = adv_df.columns.astype(str).str.strip().str.replace('\xa0', '')

        # Clean fields
        adv_df['Player'] = adv_df['Player'].astype(str).str.strip()
        adv_df['Team'] = adv_df['Team'].astype(str).str.strip()
        adv_df['Season'] = year
        adv_list.append(adv_df)

        # --- Standard passing ---
        std_files = [f for f in os.listdir(data_dir) if f"{year}_Standard_Passing" in f]
        if not std_files:
            print(f"No standard passing file found for {year}, skipping")
            continue
        std_file = std_files[0]

        std_df = pd.read_html(os.path.join(data_dir, std_file), header=0)[0]
        std_df.columns = std_df.columns.astype(str).str.strip().str.replace('\xa0', '')
        std_df['Player'] = std_df['Player'].astype(str).str.strip()
        std_df['Team'] = std_df['Team'].astype(str).str.strip()
        std_df['Season'] = year
        std_list.append(std_df)

    # --- Concatenate all years ---
    adv_all = pd.concat(adv_list, ignore_index=True)
    std_all = pd.concat(std_list, ignore_index=True)

    # --- Merge standard onto advanced ---
    merged_df = adv_all.merge(std_all, on=['Player', 'Team', 'Season'], how='left')

    # --- Extract playoff wins from QBrec ---
    def extract_wins(qbrec):
        try:
            return int(str(qbrec).split('-')[0])
        except:
            return pd.NA

    merged_df['playoff_games_won'] = merged_df['QBrec'].apply(extract_wins)

    return merged_df



In [3]:
big_df = build_advanced_dataset("data sources", start_year=2018, end_year=2024)


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)  # optional: widen display for readability

# Show first 10 rows
print(big_df.head(10))

   Rk_x             Player  Age_x Team Pos_x  G_x  GS_x  Cmp_x  Att_x    IAY  IAY/PA    CAY  CAY/Cmp  CAY/PA    YAC  YAC/Cmp  Bats  ThAwy  Spikes  Drops  Drop%  BadTh  Bad%  PktTime  Bltz  Hrry  \
0   1.0          Tom Brady   41.0  NWE    QB  3.0   3.0   85.0  125.0  837.0     6.7  459.0      5.4     3.7  494.0      5.8   0.0    1.0     0.0    8.0    6.5   21.0  16.9      2.3  24.0   3.0   
1   2.0         Jared Goff   24.0  LAR    QB  3.0   3.0   59.0  106.0  903.0     8.5  444.0      7.5     4.2  268.0      4.5   0.0    5.0     2.0    7.0    7.1   22.0  22.2      2.6  41.0  11.0   
2   3.0      Philip Rivers   37.0  LAC    QB  2.0   2.0   47.0   83.0  772.0     9.3  274.0      5.8     3.3  217.0      4.6   0.0    7.0     0.0    3.0    3.9   16.0  21.1      2.3  24.0   9.0   
3   4.0         Drew Brees   39.0  NOR    QB  2.0   2.0   54.0   78.0  613.0     7.9  281.0      5.2     3.6  269.0      5.0   0.0    2.0     1.0    2.0    2.7   14.0  18.7      2.3  10.0   6.0   
4   5.0    Patr

In [ ]:
cols_to_drop = [c for c in big_df.columns if c.endswith("_y")]
clean_df = big_df.drop(columns=cols_to_drop)

# 2. Remove _x from the remaining columns
clean_df.columns = clean_df.columns.str.replace("_x", "", regex=False)

# 3. Rename Yds.1 → YdsLost
clean_df = clean_df.rename(columns={"Yds.1": "YdsLost"})


['Rk_x', 'Player', 'Age_x', 'Team', 'Pos_x', 'G_x', 'GS_x', 'Cmp_x', 'Att_x', 'IAY', 'IAY/PA', 'CAY', 'CAY/Cmp', 'CAY/PA', 'YAC', 'YAC/Cmp', 'Bats', 'ThAwy', 'Spikes', 'Drops', 'Drop%', 'BadTh', 'Bad%', 'PktTime', 'Bltz', 'Hrry', 'Hits', 'Prss', 'Prss%', 'Scrm', 'Yds/Scr', 'Awards_x', 'Season', 'OnTgt', 'OnTgt%', 'Rk_y', 'Age_y', 'Pos_y', 'G_y', 'GS_y', 'QBrec', 'Cmp_y', 'Att_y', 'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Succ%', 'Lng', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'Sk', 'Yds.1', 'Sk%', 'NY/A', 'ANY/A', '4QC', 'GWD', 'Awards_y', 'playoff_games_won']


In [9]:
clean_df.to_csv("clean_playoff_passing.csv", index=False)
